In [14]:
import dask.dataframe as dd
import pandas as pd

def sloc_percent(row):
    row['sloc_percent'] = 100*(row.git_diff_src_churn.astype('float')/row.gh_sloc.astype('float'))
    row['gh_test_lines_per_kloc'] = row.gh_test_lines_per_kloc/1000
    return row

#df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','gh_is_pr','git_trigger_commit','gh_sloc','git_diff_src_churn','gh_diff_files_added','gh_diff_files_deleted','gh_diff_files_modified'])
df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','git_trigger_commit','gh_sloc','git_diff_src_churn', 'gh_test_lines_per_kloc','gh_diff_src_files','gh_diff_doc_files','gh_diff_other_files','git_diff_test_churn'])
pdf = df.drop_duplicates().map_partitions(sloc_percent).compute()

In [6]:
pdf_one_line_change = pdf[pdf.git_diff_src_churn == 1]
#pdf_one_line_change.sort_values('gh_sloc',inplace=True,ascending=False)
pdf_one_line_change['git_diff_src_churn'].describe()

count    16159.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: git_diff_src_churn, dtype: float64

In [9]:
print(16159/float(557211)*100) # one commit builds

2.8999786436376884


Ergebnis: 2.9% aller Ein-Commit Builds sind Ein-Zeilen Changes.

In [10]:
16159/float(680209)*100 # all builds

2.375593383798215

Ergebnis: 2.4% aller Builds sind Ein-Zeilen Changes.

##  Alle Build mit mindestens einer Änderung in den Quellen

Frage: Wieviele Source-Code-Change Builds haben nur eine geänderte Zeile.

In [17]:
pdf_greater_eq_one_line_change = pdf[pdf.git_diff_src_churn >= 1]
pdf_greater_eq_one_line_change['git_diff_src_churn'].describe()

count    5.422920e+05
mean     2.030520e+02
std      8.480826e+03
min      1.000000e+00
25%      4.000000e+00
50%      1.400000e+01
75%      6.000000e+01
max      5.278263e+06
Name: git_diff_src_churn, dtype: float64

In [21]:
(5.422920e+05/float(680209))*100

79.72432002516874

Zwischenergebnis: 80% der Builds enthalten mind. eine Src Code Zeilenänderung.

In [8]:
16159/float(5.422920e+05)*100 # all builds with greater or eq 1 src change

2.979759981707272

Ergebnis: 3% der Builds mit Sourcecode-change (>=1 Zeilen), sind Einzeilen-Änderungen

In [16]:
# Frage ist src_line-Änderung == src_file-Änderung?
pdf_one_file_change = pdf[pdf.gh_diff_src_files >= 1]
#pdf_one_line_change.sort_values('gh_sloc',inplace=True,ascending=False)
pdf_one_file_change['gh_diff_src_files'].describe()

count    580356.000000
mean          6.825200
std          71.596903
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max       37918.000000
Name: gh_diff_src_files, dtype: float64

In [20]:
(580356/float(680209))*100

85.32024715932897

In [ ]:
Zwischenergebnis: 85% der Builds enthalten mind. eine Src Code Dateiänderung.

In [18]:
580356-5.422920e+05

38064.0

Ergebnis: ca. 40k Builds mehr mit Dateiänderungen als mit Zeilenänderungen. Warum?! 

Erklärung:

1. Umbenennung einer Datei
1. Rechteänderung

## Wieviele Builds haben keine Quellcodeänderung

### Zeilenbasiert

In [11]:
680209-5.422920e+05

137917.0

In [13]:
137917.0/680209*100

20.275679974831267

Ergebnis: 20% der Builds haben keine zeilenbasierten Änderungen

### Aufteilung in Test/Doc 